In [22]:
import math
import requests
import datetime
from statistics import mean

YEAR = 2020
WEEK = 8
LI_RATIO = .1
L_SHARE = LI_RATIO
I_SHARE = 1 - LI_RATIO
TRANSFER_RATIO = 1
MARGIN_LOG = 7

In [25]:
r = requests.get('https://api.collegefootballdata.com/games?year=' + str(YEAR) + '&seasonType=regular')
p = requests.get('https://api.collegefootballdata.com/games?year=' + str(YEAR) + '&seasonType=postseason')
records = {}
games = r.json() + p.json()
for game in games:
        
    away_team = game['away_team']
    home_team = game['home_team']
    
    # With this dataset, no conference listed means they are an FCS team. 
    # I treat all FCS teams as the same, likely crappy team. 
    if not game['away_conference']:
        away_team = 'FCS Team'
    if not game['home_conference']:
        home_team = 'FCS Team'   
    
    if away_team not in records:
        records[away_team] = []
    if home_team not in records:
        records[home_team] = []
    
    if game['week'] > WEEK:
        continue
    
    # For invalid/unplayed games
    if game['away_points'] == None:
        continue
        
    margin = game['away_points'] - game['home_points']
    
    # Going to overtime counts as a one point win
    if len(game['away_line_scores']) > 4:
        margin = margin / abs(margin)
    records[away_team].append((home_team, margin))
    records[home_team].append((away_team, -margin))

In [30]:
rankings = {}
scores = {}
for team in records:
    rankings[team] = 0
    scores[team] = [0]

team_count = len(rankings.keys())

history = {} # used for checking if we have hit a repeating pattern based on ranks
opp_history = [] # used for easier lookups of what team scores were in said repeating pattern
scores = {} # {teamname: listof scores (first value is sum, subsequent scores are per each game)
repeat_point = 0
# Repeat for 1000 interations, but it should hit a repeating steady state before then
for i in range(1000):
    for team in records.keys():
        scores[team] = [0] # the first value is the total score
        games = records[team]
        for game in games:
            opponent = game[0]
            margin = game[1]
            multiplier = math.log(abs(margin))/math.log(MARGIN_LOG) + 1
            multiplier *= 1000 # just for nicer numbers
            game_score = 0
            if margin < 0:
                game_score -= L_SHARE * rankings[opponent] / team_count
                game_score -= I_SHARE / (team_count - rankings[opponent])
                if rankings[opponent] > rankings[team]:
                    game_score -= TRANSFER_RATIO * L_SHARE * (team_count - rankings[team]) / team_count
                    game_score -= TRANSFER_RATIO * I_SHARE / (1 + rankings[team])
            if margin > 0:
                game_score += L_SHARE * (team_count - rankings[opponent]) / team_count
                game_score += I_SHARE / (1 + rankings[opponent])
                if rankings[opponent] < rankings[team]:
                    game_score += TRANSFER_RATIO * L_SHARE * rankings[team] / team_count
                    game_score += TRANSFER_RATIO * I_SHARE / (team_count - rankings[team])
            scores[team] += [multiplier * game_score]
        if i > 1:
            scores[team][0] = (sum(scores[team]) + opp_history[-1][team][0]) / 2
        else:
            scores[team][0] = sum(scores[team])
    # This is sorted, but adjusting so tied teams have the same (higher) rank
    new_rank_orders = sorted(scores, key=scores.__getitem__, reverse=True)
    #print(new_rank_orders)
    new_ranks = rankings
    new_ranks[new_rank_orders[0]] = 0
    past_rank = 0
    past_score = scores[new_rank_orders[0]][0]
    for rank, team in enumerate(new_rank_orders):
        if (abs(scores[team][0] - past_score) < 0.01):
            new_ranks[team] = past_rank
        else:
            new_ranks[team] = rank
        past_rank = new_ranks[team]
        past_score = scores[team][0]
    # stop if we are repeating ourselves, record this if we are not
    # print(new_ranks['Duke'])
    if str(new_ranks) in history:
        repeat_point = history[str(new_ranks)]
        break
    else:
        history[str(new_ranks)] = i
        opp_history += [scores.copy()]
        rankings = new_ranks

# average across repeated cycled
for team in records:
    score = []
    for i in range(max(repeat_point,1), len(history)):
        score += [opp_history[i][team]]
    scores[team] = list(map(mean, zip(*score)))
    
    scores[team][0] = sum(scores[team][1:]) 
new_rank_orders = sorted(scores, key=scores.__getitem__, reverse=True)
output = ('My Computer Poll!\n'
          + 'It only takes into account games played this season, looking at margin and quality of wins and losses individually:\n\n'
          + '|Rank|Score|Team|Breakdown|\n'
          + '|-|-|-|-|\n')
for i, team in enumerate(new_rank_orders):
    W = len([x for x in records[team] if x[1] > 0])
    L = len([x for x in records[team] if x[1] < 0])
    explanation = ''
    for j in range(0, len(scores[team]) - 1):
        explanation += (str(round(scores[team][j + 1])) + ' from #'
                        + str(new_rank_orders.index(records[team][j][0]) + 1) + ' '
                        + str(records[team][j][0]) 
                        + '(' + str(records[team][j][1]) + '), ')
    line = ('|' + str(i+1) + '|' 
            + str(round(scores[team][0])) + '|' 
            + team + '(' + str(W) + '-' + str(L) + ')' + '|' 
            + explanation[0:-2] + '\n')
    output += line
print(output)
with open('Output - '
        + str(YEAR) 
          + '-' 
          + str(WEEK) + '.md', 'w') as text_file:
    text_file.write(output)

My Computer Poll!
It only takes into account games played this season, looking at margin and quality of wins and losses individually:

|Rank|Score|Team|Breakdown|
|-|-|-|-|
|1|1650|Clemson(6-0)|381 from #16 Wake Forest(24), 24 from #128 FCS Team(49), 116 from #83 Virginia(18), 856 from #4 Miami(25), 187 from #104 Georgia Tech(66), 86 from #109 Syracuse(26)
|2|1051|Alabama(5-0)|128 from #77 Missouri(19), 301 from #27 Texas A&M(28), 47 from #115 Ole Miss(15), 384 from #14 Georgia(17), 191 from #78 Tennessee(31)
|3|956|Coastal Carolina(5-0)|25 from #125 Kansas(15), 21 from #128 FCS Team(22), 339 from #22 Arkansas State(29), 363 from #7 Louisiana(3), 208 from #41 Georgia Southern(14)
|4|854|Miami(5-1)|281 from #25 UAB(17), 163 from #82 Louisville(13), 115 from #90 Florida State(42), -19 from #1 Clemson(-25), 228 from #30 Pittsburgh(12), 86 from #83 Virginia(5)
|5|758|BYU(6-0)|104 from #98 Navy(52), 151 from #76 Troy(41), 115 from #88 Louisiana Tech(31), 89 from #84 UT San Antonio(7), 256 f

In [5]:
len(history)

91

In [117]:
sorted(scores, key=opp_history[-11].__getitem__, reverse=True)

['LSU',
 'Clemson',
 'Ohio State',
 'Georgia',
 'Florida',
 'Wisconsin',
 'Penn State',
 'Oregon',
 'Oklahoma',
 'Alabama',
 'Michigan',
 'Memphis',
 'Notre Dame',
 'Auburn',
 'Baylor',
 'Minnesota',
 'Utah',
 'Iowa',
 'Cincinnati',
 'Navy',
 'Appalachian State',
 'Air Force',
 'USC',
 'Boise State',
 'Washington',
 'UCF',
 'Texas',
 'Oklahoma State',
 'Texas A&M',
 'Kansas State',
 'Iowa State',
 'Florida Atlantic',
 'SMU',
 'Arizona State',
 'Louisiana',
 'California',
 'Michigan State',
 'Indiana',
 'Tennessee',
 'Virginia',
 "Hawai'i",
 'San Diego State',
 'Oregon State',
 'Washington State',
 'Mississippi State',
 'TCU',
 'Louisville',
 'Kentucky',
 'Wake Forest',
 'North Carolina',
 'Stanford',
 'Colorado',
 'Tulane',
 'Temple',
 'BYU',
 'Missouri',
 'South Carolina',
 'Florida State',
 'Virginia Tech',
 'UCLA',
 'Pittsburgh',
 'Arizona',
 'Wyoming',
 'Texas Tech',
 'Utah State',
 'Boston College',
 'Miami',
 'Louisiana Tech',
 'Miami (OH)',
 'West Virginia',
 'Duke',
 'Nebraska'

In [29]:
for team in records:
    score = []
    for i in range(repeat_point, len(history)):
        score += [opp_history[i][team]]
    print(score)
    scores[team] = list(map(mean, zip(*score)))
    print(scores[team])
    scores[team][0] = sum(scores[team][1:])
    
new_rank_orders = sorted(scores, key=scores.__getitem__, reverse=True)
output = ''
for i, team in enumerate(new_rank_orders):
    W = len([x for x in records[team] if x[1] > 0])
    L = len([x for x in records[team] if x[1] < 0])
    explanation = ''
    for j in range(0, len(scores[team]) - 1):
        explanation += (str(round(scores[team][j + 1])) + ' from #'
                        + str(new_rank_orders.index(records[team][j][0]) + 1) + ' '
                        + str(records[team][j][0]) 
                        + '(' + str(records[team][j][1]) + '), ')
    line = ('|' + str(i+1) + '|' 
            + str(round(scores[team][0])) + '|' 
            + team + '(' + str(W) + '-' + str(L) + ')' + '|' 
            + explanation[0:-2] + '\n\n')
    output += line
print(output)

with open('Output - '
        + str(YEAR) 
          + '-' 
          + str(LI_RATIO) + '-' 
          + str(TRANSFER_RATIO) + '-' 
          + str(MARGIN_LOG) + '.txt', 'w') as text_file:
    text_file.write(output)

[[0], [6879.962042522409, -11.764678906827784, -10.74898878357421, 3129.1500681071593, 1827.087475346916, -13.740458015267174, 2068.6215613240665, -6.870229007633587, 1712.4143742160443, 2455.9836410903476, 2654.174950693832, -13.19621373317131, -14.999734322209502, -16.187682964864273], [7049.122968641201, -31.473029896043567, -85.65749728421584, 36.68305695981676, 301.5290351419339, -89.63252263447541, 357.1477002425691, -282.46885895525827, 91.91049293155214, 127.34378411528463, 397.9015359835771, -127.14447200055535, -298.7925589462954, -59.02481342030531], [7170.180994801123, -24.397243962519916, -84.21612703388041, 23.886641741276023, 292.18215118406374, -96.76037981753863, 358.53220535285504, -233.60662238524836, 95.0586055671685, 134.0726304673166, 395.2438360044893, -252.44421877535697, -298.4749469649437, -66.9604790578354], [7297.2369825023015, -21.579119822828385, -82.78117335921438, 23.886641741276023, 282.8660337481779, -96.76037981753863, 358.53220535285504, -242.5517993

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[[0], [6875.671746664828, -17.07497350911193, 1000.0, 1000.0, -18.37326235994562, -13.740458015267174, -11.764678906827784, 1564.5750340535797, 2712.414374216044, 2000.0, 2781.035935540111, 2781.035935540111, -14.999734322209502, -11.764678906827784], [7208.33064277869, -29.277887910290595, 149.79389312977102, 156.8285912560722, -91.92880588938779, -287.9007633587787, -264.3652302550419, 394.7864713789396, 363.9779770215628, 329.2768179991965, 32.60211156712781, 28.045871702041556, -62.0465200693301, -54.47473434415639], [7483.943955948664, -37.45832113247788, 150.42161052107537, 159.27682765707576, -82.79976730184646, -295.47822182308045, -273.31732793672785, 343.5330961409118, 358.4862679526776, 303.23950381679396, 21.22928195068787, 25.77136087967225, -65.74317211557413, -55.9345122692401], [7744.5606170061255, -45.679202635487485, 154.67155261420763, 160.09809254084064, -80.52919629822985, -299.3337959750174, -281.75112775912106, 349.40895124725637, 353.1893189069161, 289.961619623

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




[-30694.45996343882]
[[0], [10354.651599181385, 2796.8494399209235, -6.870229007633587, 2781.035935540111, 2674.3304101698627, 2356.207187108022, -14.999734322209502, 2633.196595377646, -11.764678906827784, -12.5525093726124, 2276.9894082696237, -13.19621373317131, 2485.3572552151813, 2764.7203321038505], [10650.12391406764, 32.787493434187496, -156.95336108447918, 390.75225704514475, 155.9175458600941, 206.47471377658954, -498.0233918119596, 70.18507464778841, -22.98735363637624, -21.523369404239396, 97.76660929580356, -419.1065731797432, 154.31646810277468, 601.338516726922], [10853.125723366187, 21.349995724587203, -164.46617170344035, 290.50945519394435, 166.0498596615347, 221.6467359804493, -495.69955342035126, 67.97368618644165, -20.172502765574045, -27.537139477659345, 97.76660929580356, -406.36090664584407, 142.57732826634407, 512.3662223008548], [11064.737870554965, 21.349995724587203, -165.33404950093555, 290.50945519394435, 171.2053992631297, 228.18313464586456, -494.683192

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [120]:
new_rank_orders

['LSU',
 'Ohio State',
 'Clemson',
 'Georgia',
 'Florida',
 'Oregon',
 'Penn State',
 'Wisconsin',
 'Oklahoma',
 'Alabama',
 'Notre Dame',
 'Memphis',
 'Michigan',
 'Baylor',
 'Auburn',
 'Minnesota',
 'Utah',
 'Iowa',
 'Cincinnati',
 'Navy',
 'Appalachian State',
 'USC',
 'Boise State',
 'Air Force',
 'Washington',
 'UCF',
 'Texas',
 'Texas A&M',
 'Kansas State',
 'Oklahoma State',
 'Iowa State',
 'SMU',
 'Florida Atlantic',
 'Arizona State',
 'California',
 'Louisiana',
 'Michigan State',
 'Indiana',
 'Tennessee',
 "Hawai'i",
 'Virginia',
 'San Diego State',
 'Oregon State',
 'Washington State',
 'Kentucky',
 'TCU',
 'UCLA',
 'Mississippi State',
 'Louisville',
 'Stanford',
 'Colorado',
 'North Carolina',
 'Wake Forest',
 'Temple',
 'Tulane',
 'South Carolina',
 'Arizona',
 'Florida State',
 'BYU',
 'Virginia Tech',
 'Wyoming',
 'Missouri',
 'Utah State',
 'Pittsburgh',
 'Boston College',
 'Texas Tech',
 'West Virginia',
 'Miami',
 'Louisiana Tech',
 'Nebraska',
 'Duke',
 'Marshall',


In [92]:
'''rankings = {}
scores = {}
for team in records:
    rankings[team] = 0
    scores[team] = 0

team_count = len(rankings.keys())

history = {} # used for checking if we have hit a repeating pattern based on ranks
opp_history = [] # used for easier lookups of what team scores were in said repeating pattern
scores = {} # {teamname: listof scores (first value is sum, subsequent scores are per each game)

# Repeat for 1000 interations, but it should hit a repeating steady state before then
for i in range(1000):
    for team in records.keys():
        scores[team] = [0] # the first value is the total score
        games = records[team]
        for game in games:
            opponent = game[0]
            margin = game[1]
            multiplier = math.log(abs(margin))/math.log(MARGIN_LOG) + 1
            multiplier *= 1000 # just for nicer numbers
            if margin < 0:
                scores[team] += [- (L_SHARE * multiplier * rankings[opponent] / team_count 
                                    + I_SHARE * multiplier * 1 / (team_count - rankings[opponent]))]
            if margin > 0:
                scores[team] += [+ (L_SHARE * multiplier * (team_count - rankings[opponent]) / team_count
                                    + I_SHARE * multiplier/ (1 + rankings[opponent]))]
        scores[team][0] = sum(scores[team])
    
    # This is sorted, but adjusting so tied teams have the same (higher) rank
    new_rank_orders = sorted(scores, key=scores.__getitem__, reverse=True)
    new_ranks = rankings
    new_ranks[new_rank_orders[0]] = 0
    past_rank = 0
    past_score = scores[new_rank_orders[0]][0]
    for rank, team in enumerate(new_rank_orders):
        if (abs(scores[team][0] - past_score) < 0.01):
            new_ranks[team] = past_rank
        else:
            new_ranks[team] = rank
        past_rank = new_ranks[team]
        past_score = scores[team][0]
    
    # stop if we are repeating ourselves, record this if we are not
    if str(new_ranks) in history:
        repeat_point = history[str(new_ranks)]
        break
    else:
        history[str(new_ranks)] = i
        opp_history += [scores.copy()]
        rankings = new_ranks

# average across repeated cycled
for team in records:
    score = []
    for i in range(repeat_point, len(history)):
        score += [opp_history[i][team]]
    scores[team] = list(map(mean, zip(*score)))
    
new_rank_orders = sorted(scores, key=scores.__getitem__, reverse=True)
output = ''
for i, team in enumerate(new_rank_orders):
    W = len([x for x in records[team] if x[1] > 0])
    L = len([x for x in records[team] if x[1] < 0])
    explanation = ''
    for j in range(1, len(scores[team])):
        explanation += (str(round(scores[team][j])) + ' from ' 
                        + str(records[team][j-1][0]) 
                        + '(' + str(records[team][j-1][1]) + ')|')
    line = ('|' + str(i+1) + '|' 
            + str(round(scores[team][0])) + '|' 
            + team + '(' + str(W) + '-' + str(L) + ')' + '|' 
            + explanation + '\n')
    output += line
print(output)

with open('Output - ' 
          + str(YEAR) + '-' 
          + str(LI_RATIO) + '-' 
          + str(I_MOD) + '-' 
          + str(MARGIN_LOG) + '.txt', 'w') as text_file:
    text_file.write(output)'''

"rankings = {}\nscores = {}\nfor team in records:\n    rankings[team] = 0\n    scores[team] = 0\n\nteam_count = len(rankings.keys())\n\nhistory = {} # used for checking if we have hit a repeating pattern based on ranks\nopp_history = [] # used for easier lookups of what team scores were in said repeating pattern\nscores = {} # {teamname: listof scores (first value is sum, subsequent scores are per each game)\n\n# Repeat for 1000 interations, but it should hit a repeating steady state before then\nfor i in range(1000):\n    for team in records.keys():\n        scores[team] = [0] # the first value is the total score\n        games = records[team]\n        for game in games:\n            opponent = game[0]\n            margin = game[1]\n            multiplier = math.log(abs(margin))/math.log(MARGIN_LOG) + 1\n            multiplier *= 1000 # just for nicer numbers\n            if margin < 0:\n                scores[team] += [- (L_SHARE * multiplier * rankings[opponent] / team_count \n     

In [ ]:
'''
records = {
    'A' : {'B':1,'C':1,'D':1,'E':1,'F':1,'G':-1},
    'B' : {'A':-1, 'C':1,'D':1,'E':1,'F':1,'G':1},
    'C' : {'A':-1,'B':-1,'D':1,'E':1,'F':1,'G':1},
    'D' : {'A':-1,'B':-1,'C':-1,'E':1,'F':1,'G':1},
    'E' : {'A':-1,'B':-1,'C':-1,'D':-1,'F':1,'G':1},
    'F' : {'A':-1,'B':-1,'C':-1,'D':-1,'E':-1,'G':1},
    'G' : {'A':1,'B':-1,'C':-1,'D':-1,'E':-1,'F':-1}
}
records = {
    'Illinois' : {'Iowa':10-19,'Minnesota':17-40,'Nebraska':38-42,'Northwestern':10-29,'Purdue':24-6,'Wisconsin':24-23},
    'Iowa' : {'Illinois':10-19, 'Minnesota':23-19,'Nebraska':27-24,'Northwestern':20-0,'Purdue':26-20,'Wisconsin':22-24},
    'Minnesota' : {'Illinois':40-17,'Iowa':19-23,'Nebraska':34-7,'Northwestern':38-22,'Purdue':38-31,'Wisconsin':17-38},
    'Nebraska' : {'Illinois':42-38,'Iowa':24-27,'Minnesota':7-34,'Northwestern':13-10,'Purdue':27-31,'Wisconsin':21-37},
    'Northwestern' : {'Illinois':29-10,'Iowa':0-20,'Minnesota':22-38,'Nebraska':10-13,'Purdue':22-24,'Wisconsin':15-24},
    'Purdue' : {'Illinois':6-24,'Iowa':20-26,'Minnesota':31-38,'Nebraska':31-27,'Northwestern':24-22,'Wisconsin':24-45},
    'Wisconsin' : {'Illinois':23-24,'Iowa':24-22,'Minnesota':38-17,'Nebraska':37-21,'Northwestern':24-15,'Purdue':45-24}
}
'''